In [ ]:
!wget https://files.grouplens.org/datasets/movielens/ml-1m.zip -P /content/drive/MyDrive/RecsysProject/
!unzip ml-1m.zip

--2025-06-27 17:59:49--  https://files.grouplens.org/datasets/movielens/ml-1m.zip
Resolving files.grouplens.org (files.grouplens.org)... 128.101.65.152
Connecting to files.grouplens.org (files.grouplens.org)|128.101.65.152|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5917549 (5.6M) [application/zip]
Saving to: ‘ml-1m.zip.1’

ml-1m.zip.1         100%[===================>]   5.64M  23.6MB/s    in 0.2s    

2025-06-27 17:59:49 (23.6 MB/s) - ‘ml-1m.zip.1’ saved [5917549/5917549]

Archive:  ml-1m.zip
   creating: ml-1m/
  inflating: ml-1m/movies.dat        
  inflating: ml-1m/ratings.dat       
  inflating: ml-1m/README            
  inflating: ml-1m/users.dat         


In [ ]:
import pandas as pd

movies= pd.read_csv("/content/drive/MyDrive/Recsys_Project/data/movies.dat" , sep = '::', encoding='latin-1' ,names = ['movieid','title','genra'] )
ratings = pd.read_csv("/content/drive/MyDrive/Recsys_Project/data/ratings.dat" , sep = '::', encoding='latin-1' ,names = ['userid','movieid','rating','timestamp'] )
users = pd.read_csv("/content/drive/MyDrive/Recsys_Project/data/users.dat" , sep = '::', encoding='latin-1' ,names = ['userid','age','occupation','zipcode'] )

/tmp/ipython-input-10-3954855700.py:3: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  movies= pd.read_csv("/content/drive/MyDrive/Recsys_Project/data/movies.dat" , sep = '::', encoding='latin-1' ,names = ['movieid','title','genra'] )
/tmp/ipython-input-10-3954855700.py:4: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  ratings = pd.read_csv("/content/drive/MyDrive/Recsys_Project/ratings.dat" , sep = '::', encoding='latin-1' ,names = ['userid','movieid','rating','timestamp'] )
/tmp/ipython-input-10-3954855700.py:5: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support re

In [ ]:
#train test split by leave 1 lasted watched movie out into test
rating_sort = ratings.sort_values(by = ['userid','timestamp'] )
rating_sort.head()

def leaveone(df):
  train = []
  test = []
  for uid,group in rating_sort.groupby('userid'):
    if len(group)<2:
      continue
    train.append(group.iloc[:-1])
    test.append(group.iloc[-1])
  train = pd.concat(train)
  test = pd.DataFrame(test)
  return train,test

In [ ]:
 train_ratings,test_ratings = leaveone(rating_sort)

In [ ]:
user_item_matrix = train_ratings.pivot(index = 'userid', columns = 'movieid',values = 'rating')
user_item_matrix.shape

(6040, 3704)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
usermovie  = user_item_matrix.fillna(0 )
user_sim = cosine_similarity(usermovie)
user_sim_df = pd.DataFrame(user_sim,index = usermovie.index,columns = usermovie.index)
user_sim_df



userid,1,2,3,4,5,6,7,8,9,10,...,6031,6032,6033,6034,6035,6036,6037,6038,6039,6040
userid,,,,,,,,,,,,,,,,,,,,,
1,1.000000,0.097901,0.123425,0.138722,0.091446,0.158977,0.060938,0.140374,0.216960,0.251341,...,0.174648,0.083457,0.070778,0.034536,0.116436,0.188828,0.138007,0.000000,0.166388,0.135600
2,0.097901,1.000000,0.153331,0.147375,0.114768,0.102242,0.303168,0.199836,0.191640,0.227801,...,0.113931,0.091828,0.269345,0.014497,0.183855,0.227456,0.207078,0.069100,0.066895,0.219155
3,0.123425,0.153331,1.000000,0.157831,0.063584,0.059256,0.140760,0.078580,0.128366,0.207762,...,0.094841,0.127645,0.163183,0.000000,0.098285,0.140586,0.108971,0.126594,0.096009,0.134677
4,0.138722,0.147375,0.157831,1.000000,0.046646,0.014139,0.135734,0.104446,0.097292,0.115675,...,0.170851,0.078462,0.361714,0.000000,0.084864,0.163640,0.131935,0.067787,0.067080,0.131550
5,0.091446,0.114768,0.063584,0.046646,1.000000,0.048026,0.127336,0.216938,0.262927,0.117366,...,0.101780,0.035917,0.061895,0.041433,0.179280,0.293644,0.173107,0.021351,0.027830,0.242056
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6036,0.188828,0.227456,0.140586,0.163640,0.293644,0.094641,0.120978,0.227874,0.236900,0.338687,...,0.132652,0.210748,0.185583,0.101653,0.267470,1.000000,0.340926,0.120882,0.216483,0.412593
6037,0.138007,0.207078,0.108971,0.131935,0.173107,0.066630,0.112698,0.144910,0.226572,0.245080,...,0.143994,0.277737,0.130254,0.120404,0.141921,0.340926,1.000000,0.051184,0.253594,0.424293
6038,0.000000,0.069100,0.126594,0.067787,0.021351,0.069285,0.000000,0.020104,0.097962,0.118852,...,0.114647,0.111931,0.042447,0.000000,0.037450,0.120882,0.051184,1.000000,0.169319,0.103706


In [ ]:
def user_cf(user_id,topkuser,topkmovie):
  simuser = user_sim_df[user_id].drop(index = user_id).sort_values(ascending=False).head(topkuser).index
  recmovie = train_ratings[train_ratings['userid'].isin(simuser) ]
  seen = set(train_ratings[train_ratings.userid == user_id].movieid)
  unseen = recmovie[~recmovie.movieid.isin(seen)]
  avg_rat = unseen.groupby('movieid')['rating'].mean().sort_values(ascending=False)
  toprec = avg_rat.head(topkmovie)
  return toprec.index.tolist()

user_cf(2,5,10)

[3744, 34, 213, 2336, 3618, 3555, 2019, 3546, 1949, 2671]

In [ ]:
#EVAL: RECALL@K
hit = 0
total = 0
for row in test_ratings.itertuples():
  user = row.userid
  true_item = row.movieid
  recs = user_cf(user,5,10)
  #print(user,true_item,recs)
  if true_item in recs:
    hit +=1
  total +=1

recallat10 = hit/total
print(recallat10)

0.02019867549668874


In [ ]:
hit,total

(122, 6040)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
